# Experiment 4: Latest features on all statistical classifiers

In [10]:
targets = [x.label for x in concatted_reviews]

In [2]:
from latest_feature_extraction import get_features_maker

## Linear Discriminant Analysis

In [58]:
get_features = get_features_maker(concatted_reviews, 775)

In [59]:
predictor_features = get_features(concatted_reviews)

Here we can only make our bag of words features so big before we crash with memr

In [60]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
import numpy
linearDA = LinearDiscriminantAnalysis()
results = cross_validate(linearDA, predictor_features.toarray(), numpy.asarray(targets), cv=2,
                         return_train_score=False)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [61]:
sum(x for x in results['test_score'])/2

0.6899020151921419

## Feature scaling

We normalise all our features to be between one and zero. We need to do this to suppress the mega features vs tiny features situation. Most classifiers use Euclidian distance, which has no knowledge of the units being used. 

In [1]:
#######################
##### Duplicated! #####
#######################

from protos import review_set_pb2, review_pb2
review_set = review_set_pb2.ReviewSet()
with open("data/yelpZip", 'rb') as f:
  review_set.ParseFromString(f.read())

In [2]:
#######################
##### Duplicated! #####
#######################

from sklearn.utils import shuffle

fake_reviews = list(filter(lambda x: x.label, review_set.reviews))
count_fake = len(fake_reviews)
genuine_reviews = []
unused_genuine_reviews = []
counter_genuine = 0
for review in shuffle(review_set.reviews):
  if review.label == True:
    continue
  if counter_genuine <= count_fake:
    genuine_reviews.append(review)
    counter_genuine += 1
  else:
    unused_genuine_reviews.append(review)
  
concatted_reviews = shuffle(fake_reviews + genuine_reviews)
print("fake:", len(fake_reviews))
print("real:", len(genuine_reviews))
print("all:", len(concatted_reviews))
print("unused real:", len(unused_genuine_reviews))

fake: 80466
real: 80467
all: 160933
unused real: 447665


In [108]:
n = 32000

Our sparse features are problematic here because we need to convert our data to array for standardisation. Since we cannot do this with sparse data without MemoryErrors, we will use dense features for our bag of words from experiment 2.

In [109]:
from latest_feature_extraction import dense_features_maker
from exp2_feature_extraction import find_words

corpus_words = [find_words(x.review_content) for x in concatted_reviews[:n]]

In [110]:
dense_features_getter = dense_features_maker(corpus_words)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [111]:
predictor_features_dense = [dense_features_getter(x) for x in corpus_words]

In [112]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(predictor_features_dense)

In [ ]:
scaled_features = scaler.transform(predictor_features_dense)

## Support Vector Machine
this takes so long to run with all the data that we could not get results for this. Instead we use a smaller data set of n=36000

In [83]:
n = 60000

In [84]:
from sklearn.svm import LinearSVC
svc = LinearSVC()

In [85]:
from latest_feature_extraction import get_features_maker # Already imported
get_features2 = get_features_maker(concatted_reviews[:n], 775)
predictor_features2 = get_features2(concatted_reviews[:n])

In [86]:
# Should already be imported
from sklearn.model_selection import cross_validate

In [87]:
targets = [x.label for x in concatted_reviews]

In [ ]:
cross_validate(svc, scaled_features, targets[:n], cv=2, return_train_score=False)

When attempting to use LDA topics to have more dense features for our SVM, we have a very low accuracy. For some reason it is ?% better to just use bag of words again.

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()

In [ ]:
cross_validate(svc, scaled_features, targets[:n], cv=2, return_train_score=False)

## K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()

In [ ]:
cross_validate(knn_classifier, scaled_features, targets[:n], cv=2, return_train_score=False)